## GRIB: using the metadata object

In [1]:
import earthkit.data as ekd
ekd.download_example_file("test6.grib")
ds = ekd.from_source("file", "test6.grib")

#### The metadata object

In [2]:
ds[0].metadata("shortName")

't'

In [3]:
md = ds[0].metadata()
md

It can be used as a dict.

In [4]:
md["shortName"], md.get("level")

('t', 1000)

<div class="alert alert-warning">
<b>Warning:</b> this object <b>does not own a GRIB handle</b> but contains a reference to the field, which provides access to the GRIB handle. Therefore if you need to store it for later use it is recommended to create a copy with override() (see below) to decouple it from the field object.</div>

#### Creating a copy of the metadata object

In [5]:
md_copy = md.override()

In [6]:
md["average"], md_copy["average"]

(279.70703560965404, 47485.4296875)

To get a copy without shrinking the GRIB handle use ``headers_only_clone=False``.

In [7]:
md_copy_full = md.override(headers_only_clone=False)

In [8]:
md_copy_full["average"]

279.70703560965404

#### Changing the metadata

In [9]:
md_mod = md.override(shortName="z", level="850")
md_mod["shortName"], md_mod["level"]

('z', 850)

Since md_mod contains a clone of the GRIB handle the original metadata did not change.

In [10]:
md["shortName"], md["level"]

('t', 1000)

#### Creating array fields from metadata and values

GRIB metadata objects play a part in building new fieldlists from (altered) values and metadata.

The following example computes the wind speed on 1000 hPa and creates a new fieldlist with a single field containing the new values. The metadata is taken from the u field and the shortName is modified.

In [11]:
from earthkit.data import FieldList
import numpy as np
u = ds.sel(param="u", level=1000)[0]
v = ds.sel(param="v", level=1000)[0]
speed = np.sqrt(u.values**2 + v.values**2)
md_speed = u.metadata().override(shortName="ws")
ds_speed = FieldList.from_array(speed, md_speed) 

In [12]:
ds_speed.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,ws,isobaricInhPa,1000,20180801,1200,0,an,0,regular_ll


In [13]:
ds_speed[0].metadata()

In [14]:
ds_speed[0].metadata("name")

'Wind speed'

In [15]:
try:
    ds_speed[0].metadata("average")
except KeyError as e:
    print(f"KeyError {e}")

KeyError 'average'


However, strictly speaking these keys do not represent metadata and they can be easily computed from the field values when needed.

In [16]:
ds_speed[0].values.mean()

7.450183054360252